In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import glob
import random
import numpy as np
from torchmetrics import Accuracy, Precision, Recall
import wandb
from pytorch_lightning.loggers import WandbLogger
import torchvision.transforms as T
from src.fire_series_dataset import FireSeriesDataset
from src.temporal_model import TemporalModel 


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

In [3]:
class FireDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size=16, img_size=224, num_workers=12):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        self.train_dataset = FireSeriesDataset(
            os.path.join(self.data_dir, "train"), self.img_size
        )
        self.val_dataset = FireSeriesDataset(
            os.path.join(self.data_dir, "val"), self.img_size
        )

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)



In [4]:
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy, Precision, Recall

class FireClassifier(pl.LightningModule):
    def __init__(self, learning_rate=1e-4, unfreeze_epoch=5):
        super(FireClassifier, self).__init__()
        self.save_hyperparameters()

        self.model = TemporalModel()

        # Initialize the accuracy, precision, and recall metrics
        self.train_accuracy = Accuracy(task="binary")
        self.val_accuracy = Accuracy(task="binary")
        self.train_precision = Precision(task="binary")
        self.val_precision = Precision(task="binary")
        self.train_recall = Recall(task="binary")
        self.val_recall = Recall(task="binary")

        # Track the epoch to unfreeze the model
        self.unfreeze_epoch = unfreeze_epoch

    def unfreeze_model(self):
        """Unfreeze efficient last 5 layers."""
        for param in self.model.efficientnet[-5:].parameters(): 
            param.requires_grad = True

            
    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()

        # Use binary cross-entropy, since the sigmoid is already applied in the model
        loss = F.binary_cross_entropy(y_hat, y.float())
        
        # Metrics computation (no need for sigmoid as it's already applied)
        acc = self.train_accuracy(y_hat, y.int())
        precision = self.train_precision(y_hat, y.int())
        recall = self.train_recall(y_hat, y.int())
        
        # Logging
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        self.log("train_precision", precision, on_step=False, on_epoch=True)
        self.log("train_recall", recall, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()

        # Use binary cross-entropy for validation as well
        loss = F.binary_cross_entropy(y_hat, y.float())
        
        # Metrics computation (no need for sigmoid as it's already applied)
        acc = self.val_accuracy(y_hat, y.int())
        precision = self.val_precision(y_hat, y.int())
        recall = self.val_recall(y_hat, y.int())
        
        # Logging
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True)
        self.log("val_precision", precision, on_step=False, on_epoch=True)
        self.log("val_recall", recall, on_step=False, on_epoch=True)
        return loss

    def on_train_epoch_end(self):
        # At the end of each training epoch, display the current metrics
        train_acc = self.train_accuracy.compute().item()
        train_precision = self.train_precision.compute().item()
        train_recall = self.train_recall.compute().item()
        print(f"Epoch {self.current_epoch}:")
        print(f"Train Accuracy: {train_acc:.4f}")
        print(f"Train Precision: {train_precision:.4f}")
        print(f"Train Recall: {train_recall:.4f}")
        
        # Unfreeze the model after the specified number of epochs
        if self.current_epoch + 1 == self.unfreeze_epoch:
            print(f"Unfreeze efficient last 5 layers at epoch {self.current_epoch + 1}")
            self.unfreeze_model()

        # Reset metrics for the next epoch
        self.train_accuracy.reset()
        self.train_precision.reset()
        self.train_recall.reset()

    def on_validation_epoch_end(self):
        # At the end of each validation epoch, display the current metrics
        val_acc = self.val_accuracy.compute().item()
        val_precision = self.val_precision.compute().item()
        val_recall = self.val_recall.compute().item()
        print(f"Validation - Epoch {self.current_epoch}:")
        print(f"Val Accuracy: {val_acc:.4f}")
        print(f"Val Precision: {val_precision:.4f}")
        print(f"Val Recall: {val_recall:.4f}")
        
        # Reset metrics for the next epoch
        self.val_accuracy.reset()
        self.val_precision.reset()
        self.val_recall.reset()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams['learning_rate'], weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss'
            }
        }


In [5]:
# Ensure wandb is finished from the previous session
wandb.finish()

# Initialize the DataModule
data_dir = "temporal_ds/images"
data_module = FireDataModule(data_dir, batch_size=16, img_size=112, num_workers=12)

# Initialize the model
model = FireClassifier(learning_rate=1e-4, unfreeze_epoch=3)

# Define callbacks
# Save the best model based on the highest recall score in validation
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor="val_recall",  # Monitor validation recall
    mode="max",            # Save the model with the highest recall
    save_top_k=1,          # Save only the best model
    dirpath="model_checkpoints/",  # Directory to save checkpoints
    filename="fire_model-{epoch:02d}-{val_recall:.4f}",  # Filename format for the saved model
    save_weights_only=True  # Save only the model weights (no optimizer state)
)

# Initialize WandbLogger
wandb_logger = pl.loggers.WandbLogger(project='fire_detection_project')

# Initialize the Trainer (total epochs include frozen + unfrozen training)
trainer = pl.Trainer(
    max_epochs=20,  # Total epochs (including frozen and unfrozen stages)
    callbacks=[checkpoint_callback],  # Add the checkpoint callback
    logger=wandb_logger
)

# Train the model
trainer.fit(model, datamodule=data_module)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mateolos. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type            | Params | Mode 
------------------------------------------------------------
0 | model           | TemporalModel   | 9.1 M  | train
1 | train_accuracy  | BinaryAccuracy  | 0      | train
2 | val_accuracy    | BinaryAccuracy  | 0      | train
3 | train_precision | BinaryPrecision | 0      | train
4 | val_precision   | BinaryPrecision | 0      | train
5 | train_recall    | BinaryRecall    | 0      | train
6 | val_recall      | BinaryRecall    | 0      | train
------------------------------------------------------------
2.6 M     Trainable params
6.5 M     Non-trainable params
9.1 M     Total params
36.564    Total estimated model params size (MB)
480       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Validation - Epoch 0:
Val Accuracy: 0.5000
Val Precision: 0.5000
Val Recall: 1.0000


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation - Epoch 18:
Val Accuracy: 0.9435
Val Precision: 0.9806
Val Recall: 0.9048
Epoch 18:
Train Accuracy: 0.9958
Train Precision: 0.9970
Train Recall: 0.9945


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


Validation - Epoch 19:
Val Accuracy: 0.9286
Val Precision: 0.9865
Val Recall: 0.8690
Epoch 19:
Train Accuracy: 0.9970
Train Precision: 0.9964
Train Recall: 0.9976


In [6]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_acc,▁▄▄▅▇▇▇█████████████
train_loss,█▆▅▄▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
train_precision,▁▃▄▅▇▇▇█████████████
train_recall,▁▂▂▄▆▇▇▇▇███████████
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
val_acc,▁▃▂▆▅▅▇▆▇▇▆█▆█▇▆▇▇█▇
val_loss,█▇▇▃▃▄▁▂▃▁▄▂▆▂▄▆▅▅▁▃
val_precision,▁▄▄▇▇▆▆▇▇▇▇▇▇▇██████
val_recall,▅▄▁▄▃▁█▄▅▆▄▇▄█▅▃▅▄▇▅
epoch,19
